# **Data Preprocessing**

In [1]:
# importing required libraries
import numpy as np
import pandas as pd
import copy


import pickle # saving and loading trained model
from os import path

# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# importing library for plotting
import matplotlib.pyplot as plt

# importing library for support vector machine classifier
from sklearn.svm import SVC
# importing library for K-neares-neighbor classifier
from sklearn.neighbors import KNeighborsClassifier
# importing library for Linear Discriminant Analysis Model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# importing library for Quadratic Discriminant Analysis Model
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn import metrics
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

from keras.layers import Dense # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.models import model_from_json # saving and loading trained model

from keras.layers import LSTM
from keras.layers import Input
from keras.models import Model

# representation of model layers
from keras.utils.vis_utils import plot_model

In [2]:
# dataset doesn't have column names, so we have to provide it
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty_level"]

In [3]:
# importing dataset
data = pd.read_csv('datasets/KDDTrain+.txt',header=None, names=col_names)

In [4]:
# print dataset
data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty_level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20
125969,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,21
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,18
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20


In [5]:
# remove attribute 'difficulty_level'
data.drop(['difficulty_level'],axis=1,inplace=True)
data.shape

(125973, 42)

In [6]:
# descriptive statistics of dataset
data.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,125973.00000,1.259730e+05,1.259730e+05,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,...,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000
mean,287.14465,4.556674e+04,1.977911e+04,0.000198,0.022687,0.000111,0.204409,0.001222,0.395736,0.279250,...,182.148945,115.653005,0.521242,0.082951,0.148379,0.032542,0.284452,0.278485,0.118832,0.120240
std,2604.51531,5.870331e+06,4.021269e+06,0.014086,0.253530,0.014366,2.149968,0.045239,0.489010,23.942042,...,99.206213,110.702741,0.448949,0.188922,0.308997,0.112564,0.444784,0.445669,0.306557,0.319459
min,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,82.000000,10.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,63.000000,0.510000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.00000,2.760000e+02,5.160000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,255.000000,255.000000,1.000000,0.070000,0.060000,0.020000,1.000000,1.000000,0.000000,0.000000
max,42908.00000,1.379964e+09,1.309937e+09,1.000000,3.000000,3.000000,77.000000,5.000000,1.000000,7479.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
# number of attack labels 
data['label'].value_counts()

normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: label, dtype: int64

In [8]:
# changing attack labels to their respective attack class
def change_label(df):
    df.label.replace(['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm'],'Dos',inplace=True)
    df.label.replace(['ftp_write','guess_passwd','httptunnel','imap','multihop','named','phf','sendmail',
    'snmpgetattack','snmpguess','spy','warezclient','warezmaster','xlock','xsnoop'],'R2L',inplace=True)
    df.label.replace(['ipsweep','mscan','nmap','portsweep','saint','satan'],'Probe',inplace=True)
    df.label.replace(['buffer_overflow','loadmodule','perl','ps','rootkit','sqlattack','xterm'],'U2R',inplace=True)

In [9]:
# calling change_label() function
change_label(data)

In [10]:
# distribution of attack classes
data.label.value_counts()

normal    67343
Dos       45927
Probe     11656
R2L         995
U2R          52
Name: label, dtype: int64

# **Data Normalization**

In [11]:
# selecting numeric attributes columns from data
numeric_col = data.select_dtypes(include='number').columns

In [12]:
# using standard scaler for normalizing
std_scaler = StandardScaler()
def normalization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

In [13]:
# data before normalization
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,Dos
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [14]:
# calling the normalization() function
data = normalization(data.copy(),numeric_col)

In [15]:
# data after normalization
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,-0.110249,tcp,ftp_data,SF,-0.007679,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.818890,-0.782367,-0.280282,0.069972,-0.289103,-0.639532,-0.624871,-0.224532,-0.376387,normal
1,-0.110249,udp,other,SF,-0.007737,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-1.035688,-1.161030,2.736852,2.367737,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,normal
2,-0.110249,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.809857,-0.938287,-0.174417,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,Dos
3,-0.110249,tcp,http,SF,-0.007723,-0.002891,-0.014089,-0.089486,-0.007736,-0.095076,...,1.258754,1.066401,-0.439078,-0.383108,0.066252,-0.572083,-0.602433,-0.387635,-0.345084,normal
4,-0.110249,tcp,http,SF,-0.007728,-0.004814,-0.014089,-0.089486,-0.007736,-0.095076,...,1.258754,1.066401,-0.439078,-0.480197,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,normal


# **One-hot-encoding**

In [16]:
# selecting categorical data attributes
cat_col = ['protocol_type','service','flag']

In [17]:
# creating a dataframe with only categorical attributes
categorical = data[cat_col]
categorical.head()

,protocol_type,service,flag
0,tcp,ftp_data,SF
1,udp,other,SF
2,tcp,private,S0
3,tcp,http,SF
4,tcp,http,SF


In [18]:
# one-hot-encoding categorical attributes using pandas.get_dummies() function
categorical = pd.get_dummies(categorical,columns=cat_col)
categorical.head()

,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,service_courier,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


# **Binary Classification**

In [19]:
# changing attack labels into two categories 'normal' and 'abnormal'
bin_label = pd.DataFrame(data.label.map(lambda x:'normal' if x=='normal' else 'abnormal'))

In [20]:
# creating a dataframe with binary labels (normal,abnormal)
bin_data = data.copy()
bin_data['label'] = bin_label

In [21]:
# label encoding (0,1) binary labels (abnormal,normal)
le1 = preprocessing.LabelEncoder()
enc_label = bin_label.apply(le1.fit_transform)
bin_data['intrusion'] = enc_label

In [22]:
np.save("le1_classes.npy",le1.classes_,allow_pickle=True)

In [23]:
# dataset with binary labels and label encoded column
bin_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,intrusion
0,-0.110249,tcp,ftp_data,SF,-0.007679,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.782367,-0.280282,0.069972,-0.289103,-0.639532,-0.624871,-0.224532,-0.376387,normal,1
1,-0.110249,udp,other,SF,-0.007737,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-1.161030,2.736852,2.367737,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,normal,1
2,-0.110249,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.938287,-0.174417,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,abnormal,0
3,-0.110249,tcp,http,SF,-0.007723,-0.002891,-0.014089,-0.089486,-0.007736,-0.095076,...,1.066401,-0.439078,-0.383108,0.066252,-0.572083,-0.602433,-0.387635,-0.345084,normal,1
4,-0.110249,tcp,http,SF,-0.007728,-0.004814,-0.014089,-0.089486,-0.007736,-0.095076,...,1.066401,-0.439078,-0.480197,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,normal,1


In [24]:
# one-hot-encoding attack label
bin_data = pd.get_dummies(bin_data,columns=['label'],prefix="",prefix_sep="") 
bin_data['label'] = bin_label
bin_data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,intrusion,abnormal,normal,label
0,-0.110249,tcp,ftp_data,SF,-0.007679,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,0.069972,-0.289103,-0.639532,-0.624871,-0.224532,-0.376387,1,0,1,normal
1,-0.110249,udp,other,SF,-0.007737,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,2.367737,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,1,0,1,normal
2,-0.110249,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,0,1,0,abnormal
3,-0.110249,tcp,http,SF,-0.007723,-0.002891,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.383108,0.066252,-0.572083,-0.602433,-0.387635,-0.345084,1,0,1,normal
4,-0.110249,tcp,http,SF,-0.007728,-0.004814,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.480197,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,1,0,1,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,-0.110249,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,0,1,0,abnormal
125969,-0.107178,udp,private,SF,-0.007744,-0.004883,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.447834,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,1,0,1,normal
125970,-0.110249,tcp,smtp,SF,-0.007382,-0.004823,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.480197,-0.289103,0.979238,-0.624871,-0.355014,-0.376387,1,0,1,normal
125971,-0.110249,tcp,klogin,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,0,1,0,abnormal


In [25]:
# selecting attributes found by using pearson correlation coefficient
numeric_bin = bin_data[['count','srv_serror_rate','serror_rate','dst_host_serror_rate','dst_host_srv_serror_rate',
                         'logged_in','dst_host_same_srv_rate','dst_host_srv_count','same_srv_rate']]

In [26]:
# joining the selected attribute with the one-hot-encoded categorical dataframe
numeric_bin = numeric_bin.join(categorical)
# then joining encoded, one-hot-encoded, and original attack label attribute
bin_data = numeric_bin.join(bin_data[['intrusion','abnormal','normal','label']])

In [27]:
# saving final dataset to disk
bin_data.to_csv("bin_data.csv")
# final dataset for binary classification
bin_data

,count,srv_serror_rate,serror_rate,dst_host_serror_rate,dst_host_srv_serror_rate,logged_in,dst_host_same_srv_rate,dst_host_srv_count,same_srv_rate,protocol_type_icmp,...,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,intrusion,abnormal,normal,label
0,-0.717045,-0.631929,-0.637209,-0.639532,-0.624871,-0.809262,-0.782367,-0.818890,0.771283,0,...,0,0,0,0,1,0,1,0,1,normal
1,-0.620982,-0.631929,-0.637209,-0.639532,-0.624871,-0.809262,-1.161030,-1.035688,-1.321428,0,...,0,0,0,0,1,0,1,0,1,normal
2,0.339648,1.605104,1.602664,1.608759,1.618955,-0.809262,-0.938287,-0.809857,-1.389669,0,...,1,0,0,0,0,0,0,1,0,abnormal
3,-0.690846,-0.184522,-0.189235,-0.572083,-0.602433,1.235694,1.066401,1.258754,0.771283,0,...,0,0,0,0,1,0,1,0,1,normal
4,-0.472521,-0.631929,-0.637209,-0.639532,-0.624871,1.235694,1.066401,1.258754,0.771283,0,...,0,0,0,0,1,0,1,0,1,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0.872361,1.605104,1.602664,1.608759,1.618955,-0.809262,-0.938287,-0.818890,-1.184947,0,...,1,0,0,0,0,0,0,1,0,abnormal
125969,-0.717045,-0.631929,-0.637209,-0.639532,-0.624871,-0.809262,0.977304,1.159389,0.771283,0,...,0,0,0,0,1,0,1,0,1,normal
125970,-0.725778,-0.631929,-0.637209,0.979238,-0.624871,1.235694,-0.893738,-0.773724,0.771283,0,...,0,0,0,0,1,0,1,0,1,normal
125971,0.523041,1.605104,1.602664,1.608759,1.618955,-0.809262,-1.094207,-0.972455,-1.366922,0,...,1,0,0,0,0,0,0,1,0,abnormal


## Gorilla Troop Optimizer (GTO) as an optimizer for training a NN

In [28]:
X = bin_data.iloc[:,0:93].values # dataset excluding target attribute (encoded, one-hot-encoded,original)
Y = bin_data[['intrusion']].values # target attribute

In [29]:
# splitting the dataset 75% for training and 25% testing
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=42)

In [30]:
def setWeights(X):
    Len=0
    for i in range(0,len(mlp.layers)):
        a=mlp.layers[i].get_weights()[0].shape
        weights=X[Len:(Len+mlp.layers[i].get_weights()[0].flatten().shape[0])]
        weights=np.reshape(weights,a) 
        Len=Len+mlp.layers[i].get_weights()[0].flatten().shape[0]
        a=mlp.layers[i].get_weights()[1].shape
        bias=X[Len:(Len+mlp.layers[i].get_weights()[1].flatten().shape[0])]
        bias=np.reshape(bias,a) 
        Len=Len+mlp.layers[i].get_weights()[1].flatten().shape[0]
        mlp.layers[i].set_weights([weights,bias])


In [34]:
def Fobj():
    mlp = Sequential() # creating model
    # adding input layer and first layer with 50 neurons
    mlp.add(Dense(units=50, input_dim=X_train.shape[1], activation='relu'))
    # output layer with sigmoid activation
    mlp.add(Dense(units=1,activation='sigmoid'))
    return mlp

In [35]:
mlp=Fobj()
# defining loss function, optimizer, metrics and then compiling model
mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

dim=0;

for i in range(len(mlp.get_weights())):
    dim = dim + mlp.get_weights()[i].flatten().shape[0]
    print(dim)

4650
4700
4750
4751


In [58]:
def GTO():
    
    D = dim   # Dimension
    P = 20    # Population 
    G = 500   # Gorillas "iterations"
    ub = 10   # Ubber bound
    lb = -10  # Lower Bound
    
    p = 0.03  
    beta = 3  
    w = 0.8

    pbest_X = np.zeros([P, D])
    pbest_F = np.zeros([P]) + np.inf
    gbest_X = np.zeros([D])
    gbest_F = np.inf
    
    Fit = np.ones(P) * np.inf
    X = np.zeros((P, D))

    arr = list()

    # initialization
    for i in range(P):
        for j in range(D):
            X[i][j] = (lb + np.random.rand(1) * (ub - lb))
        setWeights(X[i, :])
        print("------------Solution ", i+1, "-------------")
        score = mlp.evaluate(X_train, y_train, verbose=1)
        print("Accuracy:", score[1], " Loss: ", score[0])
        Fit[i] = score[0]

      
    # Update the local best position & local best fitness value
    fil = Fit < pbest_F             #  ==> for i in range(N):
                                        # ==> if Fit[i]<bF:
    pbest_X[fil] = X[fil].copy()            # ==> bPos=copy.copy(X[i]);
    pbest_F[fil] = Fit[fil].copy()          # ==> bF = Fit[i]

    # Update the global best position & global best fitness value
    if np.amin(Fit) < gbest_F:
        idx = Fit.argmin()
        gbest_X = X[idx].copy()
        gbest_F = Fit.min()

    # Main Loop 1 ---> 500
    for g in range(1, G):
        r1 = np.random.uniform()
        r2 = np.random.uniform()
        F_rule = (np.cos(2 * r1) + 1)  # Equation 3
        C = F_rule * (1 - g / G)  # Equation 2
        L = C * (2 * r2 - 1)  # Equation 4

        # Exploration Phase Loop
        for i in range(P):
            r3 = np.random.uniform()
            # Mitigate to unknown location
            if r3 < p:
                X[i] = np.random.uniform(low=lb, high=ub)  # Equation 1.1
            # Mitigate to other gorillas
            elif r3 >= 0.5:
                r4 = np.random.uniform()
                rand_X = pbest_X[np.random.randint(low=0, high=P)]
                Z = np.random.uniform(low=-C, high=C, size=[D])  # Equation 6
                H = Z * pbest_X[i]  # Equation 5
                X[i] = ((r4 - C) * rand_X + L * H)  # Equation 1.2
            # Mitigate to a known location
            else:
                r5 = np.random.uniform()
                rand_X1 = X[np.random.randint(low=0, high=P)]
                rand_X2 = X[np.random.randint(low=0, high=P)]
                X[i] = pbest_X[i] - L * (
                        L * (pbest_X[i] - rand_X1) + r5 * (pbest_X[i] - rand_X2))  # Equation 1.3 
        # Boundary processing (Clipping the X values) "Group Formation"
        X = np.clip(X, lb, ub)

        NewX = copy.copy(X[i])
        
        # Fitness value calculation after Exploration Phase
        setWeights(NewX)
        print("------------ Gorilla", g ," score after Exploration :-------------")
        score = mlp.evaluate(X_train, y_train, verbose=1)
        print("Accuracy:", score[1], " Loss: ", score[0])
        newFit = score[0]
        if newFit < Fit[i]:
            Fit[i] = newFit
            X[i] = copy.copy(NewX)

        # Update the local best position & local fitness value
        fil = Fit < pbest_F
        pbest_X[fil] = X[fil].copy()
        pbest_F[fil] = Fit[fil].copy()

        # Update the global best position & global fitness value
        if np.amin(Fit) < gbest_F:
            idx = Fit.argmin()
            gbest_X = X[idx].copy()
            gbest_F = Fit.min()


        # Exploitation Phase Loop
        for i in range(P):
            # Follow the Silverback
            if C >= w:
                g_var = 2 ** L  # Equation 9
                M = (np.abs(X.mean()) ** g_var) ** (1 / g_var)  # Equation 8
                X[i] = L * M * (pbest_X[i] - gbest_X) + pbest_X[i]  # Equation 7
            # Compete for the adult females (if C < self.w)
            else:
                r6 = np.random.uniform()
                if r6 >= 0.5:
                    E = np.random.normal(size=[D])  # Equation 13
                else:
                    E = np.random.normal()

                r7 = np.random.uniform()
                Q = (2 * r7 - 1)  # Equation 11
                A = (beta * E)  # Equation 12
                X[i] = gbest_X - (gbest_X * Q - pbest_X[i] * Q) * A  # Equation 10

        # Boundary processing (Clipping the X values) "Group Formation"
        X = np.clip(X, lb, ub)

        NewX = copy.copy(X[i])
        
        # Fitness value calculation after Exploitation Phase
        setWeights(NewX)
        print("------------ Gorilla", g ," score after Exploitation :-------------")
        score = mlp.evaluate(X_train, y_train, verbose=1)
        print("Accuracy:", score[1], " Loss: ", score[0])
        newFit = score[0]
        if newFit < Fit[i]:
            Fit[i] = newFit
            X[i] = copy.copy(NewX)

        # Update the local best position & local best fitness value
        fil = Fit < pbest_F
        pbest_X[fil] = X[fil].copy()
        pbest_F[fil] = Fit[fil].copy()

         # Update the global best position & global best fitness value
        if np.amin(Fit) < gbest_F:
            idx = Fit.argmin()
            gbest_X = X[idx].copy()
            gbest_F = Fit.min()

    print("The best so-far Fitness value obtained by GTO at iteration ", g , "==", gbest_F)
    print("\n")
        
    arr.append(gbest_X)
    arr.append(gbest_F)

    return arr


In [59]:
runs=1
avgLoss=0

for i in range(runs):
    optimizer = GTO()
    best_X = optimizer[0]
    avgLoss = avgLoss + optimizer[1] 

------------Solution  1 -------------
2953/2953 [==============================] - 1s 453us/step - loss: 309.9275 - accuracy: 0.2037
Accuracy: 0.20366430282592773  Loss:  309.9275207519531
------------Solution  2 -------------
2953/2953 [==============================] - 1s 451us/step - loss: 145.2600 - accuracy: 0.3627
Accuracy: 0.3627261221408844  Loss:  145.26004028320312
------------Solution  3 -------------
2953/2953 [==============================] - 1s 452us/step - loss: 417.0983 - accuracy: 0.2404
Accuracy: 0.24037086963653564  Loss:  417.0982666015625
------------Solution  4 -------------
2953/2953 [==============================] - 1s 447us/step - loss: 70.4774 - accuracy: 0.7700
Accuracy: 0.7699912190437317  Loss:  70.47740936279297
------------Solution  5 -------------
2953/2953 [==============================] - 1s 452us/step - loss: 262.1149 - accuracy: 0.1747
Accuracy: 0.17474782466888428  Loss:  262.11492919921875
------------Solution  6 -------------
2953/2953 [=======

2953/2953 [==============================] - 1s 468us/step - loss: 473.7343 - accuracy: 0.4292
Accuracy: 0.42915356159210205  Loss:  473.7342834472656
------------ Gorilla 11  score after Exploitation :-------------
2953/2953 [==============================] - 1s 459us/step - loss: 17.2545 - accuracy: 0.8061
Accuracy: 0.8060733079910278  Loss:  17.254518508911133
------------ Gorilla 12  score after Exploration :-------------
2953/2953 [==============================] - 1s 483us/step - loss: 1.1141 - accuracy: 0.7619
Accuracy: 0.7619259357452393  Loss:  1.1140519380569458
------------ Gorilla 12  score after Exploitation :-------------
2953/2953 [==============================] - 1s 464us/step - loss: 1.1141 - accuracy: 0.7619
Accuracy: 0.7619259357452393  Loss:  1.1140519380569458
------------ Gorilla 13  score after Exploration :-------------
2953/2953 [==============================] - 1s 459us/step - loss: 31.4849 - accuracy: 0.5361
Accuracy: 0.5360767841339111  Loss:  31.484889984

2953/2953 [==============================] - 1s 464us/step - loss: 1.2336 - accuracy: 0.6852
Accuracy: 0.6851681470870972  Loss:  1.2336286306381226
------------ Gorilla 30  score after Exploitation :-------------
2953/2953 [==============================] - 1s 461us/step - loss: 0.9671 - accuracy: 0.8452
Accuracy: 0.845193088054657  Loss:  0.9671321511268616
------------ Gorilla 31  score after Exploration :-------------
2953/2953 [==============================] - 1s 462us/step - loss: 87.7534 - accuracy: 0.1935
Accuracy: 0.1934821456670761  Loss:  87.75337219238281
------------ Gorilla 31  score after Exploitation :-------------
2953/2953 [==============================] - 1s 459us/step - loss: 0.9671 - accuracy: 0.8452
Accuracy: 0.845193088054657  Loss:  0.9671321511268616
------------ Gorilla 32  score after Exploration :-------------
2953/2953 [==============================] - 1s 460us/step - loss: 236.0978 - accuracy: 0.5708
Accuracy: 0.5708358287811279  Loss:  236.097839355468

2953/2953 [==============================] - 1s 458us/step - loss: 0.4942 - accuracy: 0.8128
Accuracy: 0.8128049373626709  Loss:  0.4941948354244232
------------ Gorilla 50  score after Exploration :-------------
2953/2953 [==============================] - 1s 461us/step - loss: 240.3451 - accuracy: 0.4256
Accuracy: 0.42558664083480835  Loss:  240.3450927734375
------------ Gorilla 50  score after Exploitation :-------------
2953/2953 [==============================] - 1s 457us/step - loss: 0.4942 - accuracy: 0.8128
Accuracy: 0.8128049373626709  Loss:  0.4941948354244232
------------ Gorilla 51  score after Exploration :-------------
2953/2953 [==============================] - 1s 464us/step - loss: 14.2218 - accuracy: 0.7967
Accuracy: 0.7967484593391418  Loss:  14.221829414367676
------------ Gorilla 51  score after Exploitation :-------------
2953/2953 [==============================] - 1s 456us/step - loss: 0.4942 - accuracy: 0.8128
Accuracy: 0.8128049373626709  Loss:  0.49419483542

2953/2953 [==============================] - 1s 456us/step - loss: 0.3843 - accuracy: 0.8192
Accuracy: 0.8191767334938049  Loss:  0.38429781794548035
------------ Gorilla 69  score after Exploration :-------------
2953/2953 [==============================] - 1s 457us/step - loss: 0.7172 - accuracy: 0.2042
Accuracy: 0.20420411229133606  Loss:  0.7172164916992188
------------ Gorilla 69  score after Exploitation :-------------
2953/2953 [==============================] - 1s 455us/step - loss: 0.3843 - accuracy: 0.8192
Accuracy: 0.8191767334938049  Loss:  0.38429781794548035
------------ Gorilla 70  score after Exploration :-------------
2953/2953 [==============================] - 1s 472us/step - loss: 65.3649 - accuracy: 0.7572
Accuracy: 0.7571841478347778  Loss:  65.3648681640625
------------ Gorilla 70  score after Exploitation :-------------
2953/2953 [==============================] - 1s 460us/step - loss: 0.3843 - accuracy: 0.8192
Accuracy: 0.8191767334938049  Loss:  0.384297817945

...

2953/2953 [==============================] - 1s 454us/step - loss: 0.2834 - accuracy: 0.9009
Accuracy: 0.9008880257606506  Loss:  0.2834417521953583
------------ Gorilla 411  score after Exploration :-------------
2953/2953 [==============================] - 1s 458us/step - loss: 102.2956 - accuracy: 0.4654
Accuracy: 0.46536266803741455  Loss:  102.2955551147461
------------ Gorilla 411  score after Exploitation :-------------
2953/2953 [==============================] - 1s 460us/step - loss: 0.2834 - accuracy: 0.9009
Accuracy: 0.9008880257606506  Loss:  0.2834417521953583
------------ Gorilla 412  score after Exploration :-------------
2953/2953 [==============================] - 1s 465us/step - loss: 4.3767 - accuracy: 0.5345
Accuracy: 0.5344573855400085  Loss:  4.376684665679932
------------ Gorilla 412  score after Exploitation :-------------
2953/2953 [==============================] - 1s 462us/step - loss: 0.2834 - accuracy: 0.9009
Accuracy: 0.9008880257606506  Loss:  0.283441752

2953/2953 [==============================] - 1s 458us/step - loss: 0.2529 - accuracy: 0.9078
Accuracy: 0.9078207612037659  Loss:  0.25290530920028687
------------ Gorilla 430  score after Exploration :-------------
2953/2953 [==============================] - 2s 501us/step - loss: 27.9420 - accuracy: 0.3629
Accuracy: 0.3629060387611389  Loss:  27.942012786865234
------------ Gorilla 430  score after Exploitation :-------------
2953/2953 [==============================] - 1s 458us/step - loss: 0.2529 - accuracy: 0.9078
Accuracy: 0.9078207612037659  Loss:  0.25290530920028687
------------ Gorilla 431  score after Exploration :-------------
2953/2953 [==============================] - 1s 459us/step - loss: 0.7023 - accuracy: 0.6466
Accuracy: 0.646588146686554  Loss:  0.7023263573646545
------------ Gorilla 431  score after Exploitation :-------------
2953/2953 [==============================] - 1s 460us/step - loss: 0.2529 - accuracy: 0.9078
Accuracy: 0.9078207612037659  Loss:  0.25290530

2953/2953 [==============================] - 1s 464us/step - loss: 0.2529 - accuracy: 0.9078
Accuracy: 0.9078207612037659  Loss:  0.25290530920028687
------------ Gorilla 449  score after Exploration :-------------
2953/2953 [==============================] - 1s 459us/step - loss: 0.2330 - accuracy: 0.9110
Accuracy: 0.9109960794448853  Loss:  0.2329740971326828
------------ Gorilla 449  score after Exploitation :-------------
2953/2953 [==============================] - 1s 462us/step - loss: 0.2330 - accuracy: 0.9110
Accuracy: 0.9109960794448853  Loss:  0.2329740971326828
------------ Gorilla 450  score after Exploration :-------------
2953/2953 [==============================] - 1s 462us/step - loss: 0.2323 - accuracy: 0.9103
Accuracy: 0.9103186726570129  Loss:  0.23230859637260437
------------ Gorilla 450  score after Exploitation :-------------
2953/2953 [==============================] - 1s 456us/step - loss: 0.2323 - accuracy: 0.9103
Accuracy: 0.9103186726570129  Loss:  0.23230859

2953/2953 [==============================] - 1s 475us/step - loss: 0.2211 - accuracy: 0.9008
Accuracy: 0.9007716178894043  Loss:  0.22106076776981354
------------ Gorilla 468  score after Exploration :-------------
2953/2953 [==============================] - 1s 469us/step - loss: 103.3214 - accuracy: 0.4654
Accuracy: 0.4653520882129669  Loss:  103.32140350341797
------------ Gorilla 468  score after Exploitation :-------------
2953/2953 [==============================] - 1s 469us/step - loss: 0.2211 - accuracy: 0.9008
Accuracy: 0.9007716178894043  Loss:  0.22106076776981354
------------ Gorilla 469  score after Exploration :-------------
2953/2953 [==============================] - 1s 471us/step - loss: 18.4332 - accuracy: 0.2408
Accuracy: 0.24079425632953644  Loss:  18.433198928833008
------------ Gorilla 469  score after Exploitation :-------------
2953/2953 [==============================] - 1s 469us/step - loss: 0.2211 - accuracy: 0.9008
Accuracy: 0.9007716178894043  Loss:  0.2210

2953/2953 [==============================] - 1s 456us/step - loss: 0.2097 - accuracy: 0.9210
Accuracy: 0.9210088849067688  Loss:  0.20970354974269867
------------ Gorilla 487  score after Exploration :-------------
2953/2953 [==============================] - 1s 458us/step - loss: 256.9242 - accuracy: 0.1583
Accuracy: 0.15828914940357208  Loss:  256.9241943359375
------------ Gorilla 487  score after Exploitation :-------------
2953/2953 [==============================] - 1s 468us/step - loss: 0.2097 - accuracy: 0.9210
Accuracy: 0.9210088849067688  Loss:  0.20970354974269867
------------ Gorilla 488  score after Exploration :-------------
2953/2953 [==============================] - 1s 460us/step - loss: 8.2702 - accuracy: 0.2347
Accuracy: 0.23470823466777802  Loss:  8.270242691040039
------------ Gorilla 488  score after Exploitation :-------------
2953/2953 [==============================] - 1s 465us/step - loss: 0.2097 - accuracy: 0.9210
Accuracy: 0.9210088849067688  Loss:  0.209703

In [60]:
best_X

array([ 0.29294103, -0.01583161,  0.07156229, ..., -0.16814033,
       -0.49304914,  0.4078607 ])

In [61]:
avgLoss

0.20970354974269867